In [1]:
import json, os, sys
%load_ext autoreload
%autoreload 2
os.environ["CUDA_VISIBLE_DEVICES"]="2"

import tensorflow as tf
from data.loader import load_detection
from utils.session_config import setup_gpus
setup_gpus(True, 0.95)

1 Physical GPUs, 1 Logical GPUs


In [2]:
import numpy as np
from matplotlib import pyplot as plt
from pycocotools.coco import COCO
train_ids = np.load("annotations/train_ids.npy")
test_ids = np.load("annotations/val_ids.npy")
coco = COCO("annotations/integrated_annotation.json")

loading annotations into memory...
Done (t=1.09s)
creating index...
index created!


In [3]:
train_ds = load_detection(train_ids, coco, "detection")
for patches, labels in train_ds.take(1):
    print(patches.shape)
    print(labels)

(19, 224, 224, 3)
tf.Tensor([0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0], shape=(19,), dtype=int32)


In [4]:
def shape_process(patches, labels):
    patches = tf.reshape(patches, [-1,224,224,3])
    labels = tf.reshape(labels, [-1])
    return patches, labels

train_ds = load_detection(train_ids, coco, "detection", shuffle=True)
train_ds = train_ds.cache().batch(8).map(shape_process).prefetch(tf.data.experimental.AUTOTUNE)
#train_ds = train_ds.cache().batch(16).map(shape_process).prefetch(tf.data.experimental.AUTOTUNE)
test_ds = load_detection(test_ids, coco, "detection")
test_ds = test_ds.batch(8).map(shape_process)

In [5]:
for patches, labels in train_ds.take(1):
    print(patches.shape)
    print(labels)

(152, 224, 224, 3)
tf.Tensor(
[0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0
 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0
 0 0 0 0], shape=(152,), dtype=int32)


# 모델 빌드 및 학습

In [6]:
model = tf.keras.models.load_model("models/detection.h5")

In [7]:
CCE = tf.keras.losses.SparseCategoricalCrossentropy()

@tf.function
def train_step(images, labels, model, optimizer, train_loss, train_acc):
    with tf.GradientTape() as tape:
        preds = model(images, training=True)
        loss = CCE(labels,preds)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_acc(labels, preds)
@tf.function
def test_step(images, labels, model, test_loss, test_acc):
    preds = model(images, training=False)
    loss = CCE(labels,preds)
    test_loss(loss)
    test_acc(labels, preds)

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')


template = "Epoch {} : detection train loss {:5f}, accuracy {:.3f}, val loss {:5f}, val accuracy {:.3f}"

optimizer= tf.keras.optimizers.Adam(learning_rate=1E-3)
EPOCH = 20

log_dir = "logs/detection"
summary_writer = tf.summary.create_file_writer(log_dir)

for epoch in range(1, EPOCH+1): 
    train_loss.reset_states()
    train_acc.reset_states()
    test_loss.reset_states()
    test_acc.reset_states()

    for i, (patches, labels) in enumerate(train_ds):
        train_step(patches, labels, model, optimizer, \
                             train_loss, train_acc)
        print("Epoch : {} train_step : {}".format(epoch, i))
    
    #print("Epoch : {}".format(train_acc.result().numpy()))
    for i, (patches, labels) in enumerate(test_ds):
        test_step(patches, labels, model, test_loss, test_acc)
    print(template.format(epoch,
                      train_loss.result().numpy(),
                      train_acc.result().numpy(),
                     test_loss.result().numpy(),
                     test_acc.result().numpy()))

    with summary_writer.as_default():
        tf.summary.scalar('train_loss', train_loss.result(), step=epoch)
        tf.summary.scalar('train_acc', train_acc.result().numpy(), step=epoch)
        tf.summary.scalar('test_loss', test_loss.result().numpy(), step=epoch)
        tf.summary.scalar('test_acc', test_acc.result().numpy(), step=epoch)

Epoch : 1 train_step : 0
Epoch : 1 train_step : 1
Epoch : 1 train_step : 2
Epoch : 1 train_step : 3
Epoch : 1 train_step : 4
Epoch : 1 train_step : 5
Epoch : 1 train_step : 6
Epoch : 1 train_step : 7
Epoch : 1 train_step : 8
Epoch : 1 train_step : 9
Epoch : 1 train_step : 10
Epoch : 1 train_step : 11
Epoch : 1 train_step : 12
Epoch : 1 train_step : 13
Epoch : 1 train_step : 14
Epoch : 1 train_step : 15
Epoch : 1 train_step : 16
Epoch : 1 train_step : 17
Epoch : 1 train_step : 18
Epoch : 1 train_step : 19
Epoch : 1 train_step : 20
Epoch : 1 train_step : 21
Epoch : 1 train_step : 22
Epoch : 1 train_step : 23
Epoch : 1 train_step : 24
Epoch : 1 train_step : 25
Epoch : 1 train_step : 26
Epoch : 1 train_step : 27
Epoch : 1 train_step : 28
Epoch : 1 train_step : 29
Epoch : 1 train_step : 30
Epoch : 1 train_step : 31
Epoch : 1 train_step : 32
Epoch : 1 train_step : 33
Epoch : 1 train_step : 34
Epoch : 1 train_step : 35
Epoch : 1 train_step : 36
Epoch : 1 train_step : 37
Epoch : 1 train_step :

In [ ]:
model.save("checkpoints/detection/model.h5")